In [3]:
import pandas as pd
from joblib import dump, load


In [4]:
df = pd.read_csv('../testing.csv')

In [5]:
df

,Type of Property,Subtype of Property,Bedrooms,Living area (m²),Kitchen type,How many fireplaces?,Terrace,Terrace surface (m²),Garden,Garden surface (m²),Surface of the plot (m²),Number of frontages,Swimming pool,Building condition,Basement,Energy class,Heating type,Kitchen type scale,Building condition scale,Region
0,apartment,other property,2,50,USA hyper equipped,1,0,50,0,150,0,0,0,As new,0,Empty,Empty,3,6,Flander


In [6]:
df = df.astype({"Living area (m²)":"float", "Terrace":"float", "Garden":"float",
                "Terrace surface (m²)":"float",
                "Garden surface (m²)":"float","Surface of the plot (m²)":"float",
                "Bedrooms":"float","How many fireplaces?":"float",
                "Subtype of Property":"object","Kitchen type":"object", 
                "Building condition":"object","Energy class":"object","Heating type":"object",
                 "Number of frontages":"float","Kitchen type scale":"float",
                 "Building condition scale":"float","Region":"object"})

In [7]:
df.dtypes

Type of Property             object
Subtype of Property          object
Bedrooms                    float64
Living area (m²)            float64
Kitchen type                 object
How many fireplaces?        float64
Terrace                     float64
Terrace surface (m²)        float64
Garden                      float64
Garden surface (m²)         float64
Surface of the plot (m²)    float64
Number of frontages         float64
Swimming pool                 int64
Building condition           object
Basement                      int64
Energy class                 object
Heating type                 object
Kitchen type scale          float64
Building condition scale    float64
Region                       object
dtype: object

In [14]:
OH_encoder = load('Encoded.pkl') # load and reuse the model

In [10]:
object_cols = ['Type of Property', 'Subtype of Property', 'Kitchen type',
       'Building condition', 'Energy class', 'Heating type', 'Region']
OH_cols = pd.DataFrame(OH_encoder.transform(df[object_cols]))
OH_cols.index = df.index
OH_cols.columns = OH_encoder.get_feature_names_out()
df = df.drop(object_cols, axis=1)
df = pd.concat([df, OH_cols], axis=1)

In [11]:
df = df.drop(['Kitchen type_Empty', 'Building condition_Empty', 
            'Energy class_Empty', 'Heating type_Empty'], axis=1)

In [12]:
filter_G = df["Garden surface (m²)"] == 0
print(filter_G)
df.loc[~filter_G,'Garden'] = 1
filter_T = df["Terrace surface (m²)"] == 0
df.loc[~filter_T,'Terrace'] = 1

0    False
Name: Garden surface (m²), dtype: bool


In [13]:
#df.to_csv("afterprocess.csv")
df

,Bedrooms,Living area (m²),How many fireplaces?,Terrace,Terrace surface (m²),Garden,Garden surface (m²),Surface of the plot (m²),Number of frontages,Swimming pool,...,Heating type_Carbon,Heating type_Electric,Heating type_Fuel oil,Heating type_Gas,Heating type_Pellet,Heating type_Solar,Heating type_Wood,Region_Brussels capital region,Region_Flander,Region_Walloon
0,2.0,50.0,1.0,1.0,50.0,1.0,150.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
reg = load('./model/Reg_model.pkl') # load and reuse the model
y_pred = reg.predict(df)
print(round(y_pred[0],2))